In [ ]:
from utils import get_current_dir
import pandas as pd

inputpath = get_current_dir().parent / "data" / "artvis_cleaned.csv"
df = pd.read_csv(inputpath)

df = df.replace("null", pd.NA)
display(df.head())


df["a.id"] = df["a.id"].astype("Int64")
df = df.dropna(subset=["a.id"])

df["a.firstname"] = df["a.firstname"].astype("string")
df["a.firstname"] = df["a.firstname"].fillna("Unknown Firstname")

df["a.lastname"] = df["a.lastname"].astype("string")
df["a.lastname"] = df["a.lastname"].fillna("Unknown Lastname")

df["a.gender"] = pd.Categorical(df["a.gender"]).add_categories("Unknown Gender")
df["a.gender"] = df["a.gender"].fillna("Unknown Gender")

df["a.birthdate"] = pd.to_datetime(df["a.birthdate"], errors="coerce")

df["a.deathdate"] = pd.to_datetime(df["a.deathdate"], errors="coerce")

df["a.birthplace"] = df["a.birthplace"].astype("string")
df["a.birthplace"] = df["a.birthplace"].fillna("Unknown Birthplace")

df["a.deathplace"] = df["a.deathplace"].astype("string")
df["a.deathplace"] = df["a.deathplace"].fillna("Unknown Deathplace")

df["a.nationality"] = df["a.nationality"].astype("string")
df["a.nationality"] = df["a.nationality"].fillna("Unknown Nationality")

df["e.id"] = df["e.id"].astype("Int64")
df = df.dropna(subset=["e.id"])

df["e.title"] = df["e.title"].astype("string")
df["e.title"] = df["e.title"].fillna("Unknown Title")

df["e.venue"] = df["e.venue"].astype("string")
df["e.venue"] = df["e.venue"].fillna("Unknown Venue")

df["e.startdate"] = df["e.startdate"].astype("Int64")
df["e.startdate"] = df["e.startdate"].fillna(0)

df["e.type"] = pd.Categorical(df["e.type"]).add_categories("Unknown Type")
df["e.type"] = df["e.type"].fillna("Unknown Type")

df["e.paintings"] = df["e.paintings"].astype("Int64")
df["e.paintings"] = df["e.paintings"].fillna(0)

df["e.country"] = df["e.country"].astype("string")
df["e.country"] = df["e.country"].fillna("Unknown Country")

df["e.city"] = df["e.city"].astype("string")
df["e.city"] = df["e.city"].fillna("Unknown City")

df["e.latitude"] = df["e.latitude"].astype("float64")
df["e.latitude"] = df["e.latitude"].fillna(0)

df["e.longitude"] = df["e.longitude"].astype("float64")
df["e.longitude"] = df["e.longitude"].fillna(0)


print(df.head())
for col in df.columns:
    print(f"column: {col}, type: {df[col].dtype}, unique values: {df[col].nunique()}, null values: {df[col].isnull().sum()}")

In [8]:
outputpath = get_current_dir().parent / "pages"

# Plotly

In [6]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

location_data = df.groupby(['e.city', 'e.latitude', 'e.longitude', 'e.startdate'])['e.paintings'].sum().reset_index()

fig = go.Figure()

fig.add_trace(
    go.Scattergeo(
        lon=location_data['e.longitude'],
        lat=location_data['e.latitude'],
        text=location_data.apply(
            lambda x: f"City: {x['e.city']}<br>"
                     f"Year: {int(x['e.startdate'])}<br>"
                     f"Paintings: {int(x['e.paintings'])}",
            axis=1
        ),
        mode='markers',
        marker=dict(
            size=location_data['e.paintings'].apply(lambda x: min(x/10, 50)),
            color=location_data['e.paintings'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Number of Paintings'),
            opacity=0.8
        ),
        name='Exhibitions'
    )
)

fig.update_layout(
    title='Art Exhibitions Over Time',
    geo=dict(
        scope='europe',
        projection_type='mercator',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)',
        showocean=True,
        oceancolor='rgb(230, 230, 250)',
        showcountries=True,
    ),
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 1000, 'redraw': True},
                               'fromcurrent': True}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                 'mode': 'immediate',
                                 'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'y': 0
    }],
    sliders=[{
        'currentvalue': {'prefix': 'Year: '},
        'pad': {'t': 50},
        'steps': [
            {
                'args': [[str(year)], {
                    'frame': {'duration': 0, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 0}
                }],
                'label': str(year),
                'method': 'animate'
            }
            for year in sorted(location_data['e.startdate'].unique())
        ]
    }]
)

frames = [
    go.Frame(
        data=[
            go.Scattergeo(
                lon=location_data[location_data['e.startdate'] == year]['e.longitude'],
                lat=location_data[location_data['e.startdate'] == year]['e.latitude'],
                text=location_data[location_data['e.startdate'] == year].apply(
                    lambda x: f"City: {x['e.city']}<br>"
                             f"Year: {int(x['e.startdate'])}<br>"
                             f"Paintings: {int(x['e.paintings'])}",
                    axis=1
                ),
                mode='markers',
                marker=dict(
                    size=location_data[location_data['e.startdate'] == year]['e.paintings'].apply(
                        lambda x: min(x/10, 50)
                    ),
                    color=location_data[location_data['e.startdate'] == year]['e.paintings'],
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title='Number of Paintings'),
                    opacity=0.8
                )
            )
        ],
        name=str(year)
    )
    for year in sorted(location_data['e.startdate'].unique())
]

fig.frames = frames

fig.write_html(str(outputpath / "plotly-v0.html"))


In [5]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

if pd.api.types.is_datetime64_any_dtype(df['e.startdate']):
    df['e.startdate'] = df['e.startdate'].astype('int64') // 10**9

fig = go.Figure()

fig.add_trace(
    go.Scattergeo(
        lon=df['e.longitude'],
        lat=df['e.latitude'],
        text=df.apply(
            lambda row: f"Exhibition: {row['e.title']}<br>"
                       f"Venue: {row['e.venue']}<br>"
                       f"City: {row['e.city']}<br>"
                       f"Paintings: {row['e.paintings']}<br>"
                       f"Year: {row['e.startdate']}",
            axis=1
        ),
        mode='markers',
        marker=dict(
            size=df['e.paintings'],
            sizeref=2.*max(df['e.paintings'])/(40.**2),
            sizemin=4,
            color=df['e.startdate'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Year')
        ),
        hoverinfo='text'
    )
)

fig.update_layout(
    title='Art Exhibitions Over Time',
    geo=dict(
        scope='europe',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)',
        showocean=True,
        oceancolor='rgb(230, 230, 250)',
        projection_type='mercator'
    ),
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': True},
                               'fromcurrent': True,
                               'transition': {'duration': 300,
                                           'easing': 'quadratic-in-out'}}],
                'label': '▶',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                 'mode': 'immediate',
                                 'transition': {'duration': 0}}],
                'label': '⏸',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'y': 0
    }]
)

frames = [
    go.Frame(
        data=[
            go.Scattergeo(
                lon=df[df['e.startdate'] == year]['e.longitude'],
                lat=df[df['e.startdate'] == year]['e.latitude'],
                text=df[df['e.startdate'] == year].apply(
                    lambda row: f"Exhibition: {row['e.title']}<br>"
                               f"Venue: {row['e.venue']}<br>"
                               f"City: {row['e.city']}<br>"
                               f"Paintings: {row['e.paintings']}<br>"
                               f"Year: {year}",
                    axis=1
                ),
                mode='markers',
                marker=dict(
                    size=df[df['e.startdate'] == year]['e.paintings'],
                    sizeref=2.*max(df['e.paintings'])/(40.**2),
                    sizemin=4,
                    color=year,
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title='Year')
                ),
                hoverinfo='text'
            )
        ],
        name=f'frame_{year}'
    ) for year in sorted(df['e.startdate'].unique())
]

fig.frames = frames

fig.update_layout(
    sliders=[{
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Year: ',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': [
            {
                'args': [[f'frame_{year}'],
                        {'frame': {'duration': 300, 'redraw': True},
                         'mode': 'immediate',
                         'transition': {'duration': 300}}],
                'label': str(year),
                'method': 'animate'
            } for year in sorted(df['e.startdate'].unique())
        ]
    }]
)

fig.write_html(str(outputpath / "plotly-v1.html"))


In [4]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()

fig.add_trace(
    go.Scattergeo(
        lon=df['e.longitude'],
        lat=df['e.latitude'],
        text=df.apply(
            lambda row: f"Exhibition: {row['e.title']}<br>"
                       f"Venue: {row['e.venue']}<br>"
                       f"City: {row['e.city']}<br>"
                       f"Paintings: {row['e.paintings']}<br>"
                       f"Year: {int(row['e.startdate'])}",
            axis=1
        ),
        mode='markers',
        marker=dict(
            size=df['e.paintings'],
            sizeref=2.*max(df['e.paintings'])/(40.**2),
            sizemin=4,
            color=df['e.startdate'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Year')
        ),
        hoverinfo='text'
    )
)

fig.update_layout(
    title='Art Exhibitions Over Time',
    geo=dict(
        scope='europe',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)',
        showocean=True,
        oceancolor='rgb(230, 230, 250)',
        projection_type='mercator'
    ),
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': True},
                               'fromcurrent': True,
                               'transition': {'duration': 300,
                                           'easing': 'quadratic-in-out'}}],
                'label': '▶',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                 'mode': 'immediate',
                                 'transition': {'duration': 0}}],
                'label': '⏸',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'y': 0
    }],
    sliders=[{
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Year: ',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': [
            {
                'args': [[f'frame_{year}'],
                        {'frame': {'duration': 300, 'redraw': True},
                         'mode': 'immediate',
                         'transition': {'duration': 300}}],
                'label': str(year),
                'method': 'animate'
            } for year in sorted(df['e.startdate'].unique())
        ]
    }]
)

frames = [
    go.Frame(
        data=[
            go.Scattergeo(
                lon=df[df['e.startdate'] == year]['e.longitude'],
                lat=df[df['e.startdate'] == year]['e.latitude'],
                text=df[df['e.startdate'] == year].apply(
                    lambda row: f"Exhibition: {row['e.title']}<br>"
                               f"Venue: {row['e.venue']}<br>"
                               f"City: {row['e.city']}<br>"
                               f"Paintings: {row['e.paintings']}<br>"
                               f"Year: {int(row['e.startdate'])}",
                    axis=1
                ),
                mode='markers',
                marker=dict(
                    size=df[df['e.startdate'] == year]['e.paintings'],
                    sizeref=2.*max(df['e.paintings'])/(40.**2),
                    sizemin=4,
                    color=df[df['e.startdate'] == year]['e.startdate'],
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title='Year')
                ),
                hoverinfo='text'
            )
        ],
        name=f'frame_{year}'
    ) for year in sorted(df['e.startdate'].unique())
]

fig.frames = frames

fig.write_html(str(outputpath / "plotly-v2.html"))

In [9]:
import plotly.express as px

df['e.startdate'] = pd.to_datetime(df['e.startdate'], format='%Y')

fig = px.scatter_geo(df,
                    lat='e.latitude',
                    lon='e.longitude',
                    color='e.type',
                    size='e.paintings',
                    hover_name='e.title',
                    hover_data={
                        'e.venue': True,
                        'e.city': True,
                        'e.country': True,
                        'e.paintings': True,
                        'e.latitude': False,
                        'e.longitude': False
                    },
                    animation_frame=df['e.startdate'].dt.year,
                    title='Art Exhibition Locations Over Time',
                    projection='natural earth',
                    size_max=30)

fig.update_geos(
    showcoastlines=True,
    coastlinecolor="Black",
    showland=True,
    landcolor="LightGray",
    showocean=True,
    oceancolor="LightBlue",
    showcountries=True,
    countrycolor="Gray"
)

fig.update_layout(
    title_x=0.5,
    geo=dict(
        scope='world',
        showframe=False,
        projection_type='equirectangular'
    )
)

fig.write_html(str(outputpath / "plotly-v3.html"))